In [1]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import column,row
from bokeh.models import Div
from bokeh.palettes import Spectral6

import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select"

output_notebook();

Loading BokehJS ...

# chifflot-8.lille

- PowerEdge R740
- Intel(R) Xeon(R) Gold 6126 CPU @ 2.60GHz
- L1 cache: size: 768KiB
- L2 cache: size: 12MiB
- L3 cache: size: 19MiB
- Memory: size: 192GiB, capabilities: ecc, DR4 Synchronous Registered (Buffered) 2666 MHz (0.4 ns)
- pci: 3.0 16x 16 GB/s
- Tesla V100-PCIE-32GB 
- Driver Version: 550.54.14
- CUDA Version: 12.4

## Data sources - Performance

In [3]:
data_chifflot8_cpu = json.load(open('chifflot-8/results_cpu.json', 'r'))
labels_chifflot8_cpu = [key for key, value in data_chifflot8_cpu.items()]

data_chifflot8_gpu = json.load(open('chifflot-8/results_gpu.json', 'r'))
labels_chifflot8_gpu = [key for key, value in data_chifflot8_gpu.items()]

data_chifflot8_sycl_cpu = json.load(open('chifflot-8/results_sycl_cpu.json', 'r'))
labels_chifflot8_sycl_cpu = [key for key, value in data_chifflot8_sycl_cpu.items()]

data_chifflot8_sycl_gpu = json.load(open('chifflot-8/results_sycl_gpu.json', 'r'))
labels_chifflot8_sycl_gpu = [key for key, value in data_chifflot8_sycl_gpu.items()]

### CPU

In [8]:
# Intel Xeon Gold 6126 (Skylake), x86_64, 2.60GHz, 2 CPUs/node, 12 cores/CPU

colors = ['green', 'red', 'blue', 'darkturquoise', 'coral']
i = 0

p1 = figure(title="Time", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'N'
p1.yaxis.axis_label = 'Time ( ms )'
p2.xaxis.axis_label = 'N'
p2.yaxis.axis_label = 'MSE'

errors_total = list()

for label in labels_chifflot8_cpu:
    times = list()
    errors = list()
    data_err = data_chifflot8_cpu[label]
    sizes = [key for key, value in data_err.items()]
    for size in sizes:
        times.append(data_err[size]['time'])
        errors.append(data_err[size]['error'])
    sizes = list(map(int, sizes))
    
    data_plot = pd.DataFrame({'size':sizes, 'time':times, 'error':errors})
    
    errors_total += [errors]
    errors = list(map(float, errors))
    errors_t = dict()
    errors_t['sizes'] = sizes
    for key, value in zip(labels_chifflot8_cpu, errors_total):
        errors_t[key] = value
        
    source_chifflot8_cpu_errors = ColumnDataSource(pd.DataFrame(errors_t))
    
    data_plot['error_low'] = data_plot['time'] - np.array(errors)
    data_plot['error_high'] = data_plot['time'] + np.array(errors)
    data_plot = data_plot.sort_values(by=['size'], ascending=True)
    data_plot = data_plot.reset_index(drop=True)
    
    source_chifflot8_cpu = ColumnDataSource(data_plot)
    
    p1.square(source=source_chifflot8_cpu, x='size', y='time', legend_label=labels_chifflot8_cpu[i], fill_color=None, line_color=colors[i])
    p1.line(source=source_chifflot8_cpu, x='size', y='time', line_color=colors[i])
    
    if label != 'serial':
        p2.square(source=source_chifflot8_cpu_errors, x='sizes', y=label, legend_label=labels_chifflot8_cpu[i], color=colors[i])
    
    if label == 'mkl':
        data_mkl_chifflot8 = data_plot.copy(deep=True)
        error_mkl_chifflot8 = pd.DataFrame(errors_t)
        
    if label == 'openMP':
        data_openMP_chifflot8 = data_plot.copy(deep=True)
        error_openMP_chifflot8 = pd.DataFrame(errors_t)
        
    i=i+1

maxi = max(max([sublist[:] for sublist in errors_total]))

p2.y_range.end = maxi*1.1
p1.legend.location = "top_left"
p2.legend.location = "top_left"

#show(gridplot([p1, p2], ncols=2))
show(column(Div(text=" Intel Xeon Gold 6126 (Skylake), x86_64, 2.60GHz, 2 CPUs/node, 12 cores/CPU"), row(p1, p2)))

In [10]:
# SyCL
i = 0

colors = ['green', 'red', 'blue', 'black', 'coral']
p1 = figure(title="Time", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'


for label in labels_chifflot8_sycl_cpu:
    times = list()
    errors = list()
    data_err = data_chifflot8_sycl_cpu[label]
    sizes = [key for key, value in data_err.items()]
    for size in sizes:
        times.append(data_err[size]['time'])
        errors.append(data_err[size]['error'])
    sizes = list(map(int, sizes))

    data_plot = pd.DataFrame({'size':sizes, 'time':times, 'error':errors})
    # Add error bars to the DataFrame
    errors = list(map(float, errors))
    data_plot['error_low'] = data_plot['time'] - np.array(errors)
    data_plot['error_high'] = data_plot['time'] + np.array(errors)
    data_plot = data_plot.sort_values(by=['size'], ascending = True)
    data_plot = data_plot.reset_index(drop=True)
    
    source_sycl_chifflot8 = ColumnDataSource(data_plot)
    
    p1.square(source=source_sycl_chifflot8, x='size', y='time', legend_label=labels_chifflot8_sycl_cpu[i], fill_color=None, line_color='darkturquoise')
    p1.line(source=source_sycl_chifflot8, x='size', y='time', line_color='darkturquoise')
    
    p1.square(source=source_mkl_chifflot8, x='size', y='time', legend_label='mkl', fill_color=None, line_color='blue')
    p1.line(source=source_mkl_chifflot8, x='size', y='time', line_color='blue')
    
    i=i+1


show(column(Div(text="Intel Xeon Gold 6126 (Skylake), x86_64, 2.60GHz, 2 CPUs/node, 12 cores/CPU"), row(p1)))